In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import probnum as pn

import linpde_gp

In [ ]:
import experiment_utils
from experiment_utils import config

config.experiment_name = "0000_cpu_stationary_1d"
config.target = "jmlr"
config.debug_mode = True

In [ ]:
%matplotlib inline

import matplotlib.axes
import matplotlib.ticker

plt.rcParams.update(config.tueplots_bundle())

class BeliefPlotter:
    def __init__(self, pde: linpde_gp.problems.pde.PoissonEquation):
        self._pde = pde

        self._plt_grid = np.linspace(*self._pde.domain, 100)

    @staticmethod
    def _build_cond_events_str(
        conditioned_on: list[str],
        simplified_labels: bool = False,
    ) -> str:
        for key in conditioned_on:
            match key:
                case "bc":
                    yield (
                        r"\mathrm{BC}"
                        if simplified_labels
                        else r"u\vert_{\partial \Omega} = g"
                    )
                case "pde":
                    yield (
                        r"\mathrm{PDE}"
                        if simplified_labels
                        else r"-k \Delta u(x_{\mathrm{PDE}, i}) - \dot{q}_V(x_{\mathrm{PDE}, i}) = 0"
                    )
                case "dts":
                    yield (
                        r"\mathrm{DTS}"
                        if simplified_labels
                        else r"u(x_{\mathrm{DTS}, i}) = u_{\mathrm{DTS}, i}"
                    )
                case "nbc":
                    yield (
                        r"\mathrm{NBC}"
                        if simplified_labels
                        else r"-k \nabla_n u\vert_{\partial \Omega} - \dot{q}_A = 0"
                    )
                case _:
                    raise ValueError(f"Unknown event '{key}'")

    @staticmethod
    def _configure_axes(ax: matplotlib.axes.Axes, plt_grid: np.ndarray):
        ax.xaxis.set_major_locator(
            matplotlib.ticker.FixedLocator(
                [
                    plt_grid[0],
                    plt_grid[-1],
                ]
            )
        )
        ax.xaxis.set_major_formatter(
            matplotlib.ticker.FixedFormatter(
                [
                    rf"${plt_grid[0]:.0f}\,\mathrm{{mm}}$",
                    r"$l_{\mathrm{CPU}}$",
                ]
            )
        )

        ax.xaxis.set_minor_locator(matplotlib.ticker.AutoMinorLocator())
        ax.xaxis.set_minor_formatter(
            matplotlib.ticker.FormatStrFormatter(r"$%d\,\mathrm{mm}$")
        )

    def plot_belief(
        self,
        ax,
        u: pn.randprocs.GaussianProcess,
        conditioned_on: list[str] = [],
        X_pde: np.ndarray | None = None,
        boundary_condition: linpde_gp.problems.pde.DirichletBoundaryCondition | None = None,
        solution: pn.Function | None = None,
        X_dts: np.ndarray | None = None,
        u_X_dts: pn.randvars.RandomVariable | None = None,
        simplified_labels: bool = False,
    ):
        cond_events_str = ", ".join(
            BeliefPlotter._build_cond_events_str(
                conditioned_on,
                simplified_labels=simplified_labels
            )
        )

        u_label = (
            fr"$u \mid {cond_events_str}$"
            if len(cond_events_str) > 0
            else "$u$"
        )
        
        u.plot(
            ax,
            self._plt_grid,
            num_samples=10,
            rng=np.random.default_rng(24),
            color="C0",
            label=u_label,
        )
        
        if solution is not None:
            ax.plot(
                self._plt_grid,
                solution(self._plt_grid),
                color="C1",
                label="$u^*$",
            )
        
        for key in conditioned_on:
            if key == "bc":
                g = pn.randvars.asrandvar(boundary_condition.values)

                ax.errorbar(
                    self._pde.domain.boundary,
                    g.mean,
                    yerr=1.96 * g.std,
                    fmt="+",
                    capsize=2,
                    color="C2",
                    label=r"$g$",
                )
            elif key == "pde":
                for i, x in enumerate(X_pde):
                    label = None

                    if i == 0:
                        if simplified_labels:
                            label = r"$x_{\mathrm{PDE}, i}$"
                        else:
                            label = (
                                fr"$x_{{\mathrm{{PDE}}, 1}}, \dots, "
                                fr" x_{{\mathrm{{PDE}}, {X_pde.shape[0]}}}$"
                            )

                    ax.axvline(
                        x,
                        color="C3",
                        alpha=0.1,
                        linestyle="--",
                        label=label,
                    )
            elif key == "dts":
                u_X_dts = pn.asrandvar(u_X_dts)

                if simplified_labels:
                    label = r"$(x_{\mathrm{DTS}, i}, u_{\mathrm{DTS}, i})$"
                else:
                    label = (
                        r"$u_{{\mathrm{{DTS}}, 1}}, \dots, "
                        fr"u_{{\mathrm{{DTS}}, {X_dts.shape[0]}}}$"
                    )

                ax.errorbar(
                    X_dts,
                    u_X_dts.mean,
                    yerr=1.96 * u_X_dts.std,
                    fmt="+",
                    capsize=2,
                    color="C4",
                    label=label,
                )
            elif key == "nbc":
                neumann_bc = pn.asrandvar(boundary_condition)

                linpde_gp.utils.plotting.plot_local_taylor_processes(
                    ax,
                    xs=self._pde.domain.boundary,
                    coeffs_xs=[
                        pn.randvars.Normal(
                            np.array(
                                [
                                    u.mean(self._pde.domain[idx]),
                                    (
                                        -neumann_bc.mean[idx]
                                        if idx == 0
                                        else neumann_bc.mean[idx]
                                    ),
                                ],
                            ),
                            np.diag(
                                [
                                    0.0,
                                    neumann_bc.var[idx],
                                ],
                            ),
                        )
                        for idx in [0, 1]
                    ],
                    radius=0.3,
                    color="C5",
                    rel_fill_alpha=0.25,
                    label=r"$\dot{q}_A$",
                )
        
        BeliefPlotter._configure_axes(ax, self._plt_grid)

        ax.yaxis.set_major_formatter(
            matplotlib.ticker.FormatStrFormatter(r"$%d \, {}^{\circ} \mathrm{C}$")
        )

        ax.legend()

    def plot_pred_belief(
        self,
        ax,
        u: pn.randprocs.GaussianProcess,
        conditioned_on: list[str] = [],
        X_pde: np.ndarray | None = None,
        f_X_pde: pn.randvars.RandomVariable | None = None,
        simplified_labels: bool = False,
    ):
        cond_events_str = ", ".join(
            BeliefPlotter._build_cond_events_str(
                conditioned_on,
                simplified_labels=simplified_labels
            )
        )
        
        u_label = (
            fr"$-k \Delta u \mid {cond_events_str}$"
            if len(cond_events_str) > 0
            else r"$-k \Delta u$"
        )

        self._pde.diffop(u).plot(
            ax,
            self._plt_grid,
            num_samples=10,
            rng=np.random.default_rng(24),
            color="C0",
            label=u_label,
        )
        
        self._pde.rhs.plot(
            ax,
            self._plt_grid,
            color="C1",
            label="$\dot{q}_V$",
        )
        
        if "pde" in conditioned_on:
            f_X_pde = pn.randvars.asrandvar(f_X_pde)

            if simplified_labels:
                label = r"$(x_{\mathrm{PDE}, i}, \dot{q}_V(x_{\mathrm{PDE}, i}))$"
            else:
                label = (
                    r"$\dot{q}_V(x_{{\mathrm{{PDE}}, 1}}), \dots, "
                    fr"\dot{{q}}_V(x_{{\mathrm{{PDE}}, {X_pde.shape[0]}}})$"
                )

            ax.errorbar(
                X_pde,
                f_X_pde.mean,
                yerr=1.96 * f_X_pde.std,
                fmt="+",
                capsize=2,
                c="C3",
                label=label,
            )
        
        BeliefPlotter._configure_axes(ax, self._plt_grid)

        ax.yaxis.set_major_formatter(
            matplotlib.ticker.FormatStrFormatter(
                r"$%d \, \frac{\mathrm{W}}{\mathrm{mm}^3}$"
            )
        )

        ax.legend()

## Problem Definition

### References
1. https://en.wikichip.org/wiki/intel/microarchitectures/coffee_lake#Quad-Core
2. https://ark.intel.com/content/www/us/en/ark/products/134896/intel-core-i59600k-processor-9m-cache-up-to-4-60-ghz.html
3. https://ark.intel.com/content/www/us/en/ark/products/134599/intel-core-i912900k-processor-30m-cache-up-to-5-20-ghz.html
4. https://journals.aps.org/pr/pdf/10.1103/PhysRev.134.A1058

### Geometry

In [ ]:
w_CPU = 9.19  # mm, [1]
l_CPU = 16.0  # ≈ 16.28 mm, [1]
d_CPU = 0.37  # mm, datasheed linked under [3] ("Supplemental Information / Datasheet")

A_CPU_top = l_CPU * w_CPU  # mm^2
A_CPU_EW = w_CPU * d_CPU  # mm^2
A_CPU_NS = l_CPU * d_CPU  # mm^2
A_CPU_TI = A_CPU_top + 2 * A_CPU_EW + 2 * A_CPU_NS  # mm^2, thermal interface area

V_CPU = w_CPU * l_CPU * d_CPU  # mm^3

In [ ]:
domain = linpde_gp.domains.asdomain([0, l_CPU])

In [ ]:
X_core_centers = np.array([0.2, 0.4, 0.6]) * l_CPU  # estimated from the schematic in [1]

### Material Properties

In [ ]:
# Thermal conductivity
k_CPU = 1.56 # W/cm K, [4] TODO: Improve this estimate
k_CPU *= 10  # W/mm K

### Heat Sources and Heat Sinks

In [ ]:
TDP_CPU = 95 # W, [2]

In [ ]:
# CPU Cores = Heat Sources
core_heat_std = 0.09 * l_CPU

f_CPU = TDP_CPU / (w_CPU * d_CPU) * pn.LambdaFunction(
    lambda x: (
        1.0 / (np.sqrt(2 * np.pi) * core_heat_std)
        * np.mean(
            np.exp(-0.5 * ((x[..., None] - X_core_centers) / core_heat_std) ** 2),
            axis=-1,
        )
    ),
    input_shape=(),
    output_shape=(),
)

In [ ]:
# CPU cooler = "Uniform Heat Sink"
f_cooler = linpde_gp.functions.Constant(
    input_shape=(),
    value=TDP_CPU / A_CPU_top / d_CPU,
)

In [ ]:
# Total amount of heat entering/leaving the CPU
f = f_CPU - f_cooler

### Visualization (Geometry and RHS)

In [ ]:
with plt.rc_context(config.tueplots_bundle(nrows=2, rel_width=0.5)):
    fig, ax = plt.subplots(nrows=2, sharex=True)

    # CPU Schematic
    ax[0].add_patch(
        plt.Rectangle(
            (0, 0),
            l_CPU,
            w_CPU,
            edgecolor="black",
            facecolor="white",
            linewidth=0.5,
        )
    )

    for x_core_center in X_core_centers:
        core_width = 2.5
        core_height = 0.45 * w_CPU

        ax[0].add_patch(
            plt.Rectangle(
                (x_core_center - core_width / 2, 0),
                core_width,
                core_height,
                edgecolor="black",
                facecolor="white",
                linewidth=0.5,
            )
        )

        ax[0].text(
            x_core_center,
            core_height / 2,
            "CPU Core",
            rotation="vertical",
            horizontalalignment="center",
            verticalalignment="center",
            fontsize="small",
        )

        ax[0].add_patch(
            plt.Rectangle(
                (x_core_center - core_width / 2, w_CPU - core_height),
                core_width,
                core_height,
                edgecolor="black",
                facecolor="white",
                linewidth=0.5,
            )
        )

        ax[0].text(
            x_core_center,
            w_CPU - core_height / 2,
            "CPU Core",
            rotation="vertical",
            horizontalalignment="center",
            verticalalignment="center",
            fontsize="small",
        )

    ax[0].plot([0.0, l_CPU], 2 * [core_height / 2], c="C1")

    ax[0].set_ylim(-0.5, w_CPU + 0.5)
    ax[0].yaxis.set_major_locator(
        matplotlib.ticker.FixedLocator(
            [
                0.0,
                w_CPU,
            ]
        )
    )
    ax[0].yaxis.set_major_formatter(
        matplotlib.ticker.FixedFormatter(
            [
                rf"$0\,\mathrm{{mm}}$",
                r"$w_{\mathrm{CPU}}$",
            ]
        )
    )

    ax[0].yaxis.set_minor_locator(matplotlib.ticker.AutoMinorLocator())
    ax[0].yaxis.set_minor_formatter(
        matplotlib.ticker.FormatStrFormatter(r"$%d\,\mathrm{mm}$")
    )

    # for spine in ax[0].spines.values():
    #     spine.set_visible(False)

    # ax[0].xaxis.set_visible(False)
    # ax[0].yaxis.set_visible(False)

    # PDE RHS
    plt_grid = np.linspace(*domain, 100)

    f.plot(
        ax[1],
        plt_grid,
        c="C1",
        label=r"$\dot{q}_V$"
    )

    BeliefPlotter._configure_axes(ax[1], plt_grid)
    ax[1].yaxis.set_major_formatter(
        matplotlib.ticker.FormatStrFormatter(
            r"$%d \, \frac{\mathrm{W}}{\mathrm{mm}^3}$"
        )
    )
    ax[1].legend(loc="lower center")

experiment_utils.savefig("00_geometry_pde_rhs")

### PDE

In [ ]:
pde = linpde_gp.problems.pde.PoissonEquation(
    domain=domain,
    rhs=f,
    alpha=k_CPU,
)

plotter = BeliefPlotter(pde)

# Prior

In [ ]:
u_prior = pn.randprocs.GaussianProcess(
    mean=linpde_gp.functions.Constant(input_shape=(), value=59.0),
    cov=3.0 ** 2 * linpde_gp.randprocs.kernels.Matern(
        input_shape=(),
        p=3,
        lengthscale=8.0,
    ),
)

In [ ]:
plotter.plot_belief(
    ax=plt.gca(),
    u=u_prior,
)

In [ ]:
plotter.plot_pred_belief(
    ax=plt.gca(),
    u=u_prior,
)

In [ ]:
with plt.rc_context(config.tueplots_bundle(nrows=2, rel_width=0.5)):
    with plt.rc_context({"legend.loc": "lower center"}):
        fig, ax = plt.subplots(nrows=2, sharex=True)

        plotter.plot_belief(
            ax=ax[0],
            u=u_prior,
            simplified_labels=True,
        )

        plotter.plot_pred_belief(
            ax=ax[1],
            u=u_prior,
            simplified_labels=True,
        )

experiment_utils.savefig("00_prior")

## Posterior (PDE First)

### Conditioning on the PDE

In [ ]:
X_pde = np.linspace(pde.domain[0] + 0.5, pde.domain[1] - 0.5, 15)
f_X_pde = pde.rhs(X_pde)

In [ ]:
u_cond_pde = u_prior.condition_on_observations(
    X=X_pde,
    Y=np.zeros_like(X_pde),
    L=pde.diffop,
    b=-pn.randvars.asrandvar(f_X_pde),
)

In [ ]:
plotter.plot_belief(
    ax=plt.gca(),
    u=u_cond_pde,
    conditioned_on=["pde"],
    X_pde=X_pde,
)

In [ ]:
plotter.plot_pred_belief(
    ax=plt.gca(),
    u=u_cond_pde,
    conditioned_on=["pde"],
    X_pde=X_pde,
    f_X_pde=f_X_pde,
)

In [ ]:
with plt.rc_context(config.tueplots_bundle(nrows=2, rel_width=0.5)):
    with plt.rc_context({"legend.loc": "lower center"}):
        fig, ax = plt.subplots(nrows=2, sharex=True)

        plotter.plot_belief(
            ax=ax[0],
            u=u_cond_pde,
            conditioned_on=["pde"],
            X_pde=X_pde,
            simplified_labels=True,
        )

        plotter.plot_pred_belief(
            ax=ax[1],
            u=u_cond_pde,
            conditioned_on=["pde"],
            X_pde=X_pde,
            f_X_pde=f_X_pde,
            simplified_labels=True,
        )

experiment_utils.savefig("01_cond_pde")

### Conditioning on Exact Dirichlet Boundary Conditions

In [ ]:
g = np.array([60.5, 58.5])

bvp = linpde_gp.problems.pde.PoissonEquationDirichletProblem(pde, boundary_values=g)

In [ ]:
u_cond_pde_bc = u_cond_pde.condition_on_observations(
    X=np.asarray(pde.domain.boundary),
    Y=g,
)

In [ ]:
plotter.plot_belief(
    ax=plt.gca(),
    u=u_cond_pde_bc,
    conditioned_on=["pde", "bc"],
    X_pde=X_pde,
    boundary_condition=bvp.boundary_conditions[0],
)

In [ ]:
plotter.plot_pred_belief(
    ax=plt.gca(),
    u=u_cond_pde_bc,
    conditioned_on=["pde", "bc"],
    X_pde=X_pde,
    f_X_pde=f_X_pde,
)

In [ ]:
with plt.rc_context(config.tueplots_bundle(nrows=2, rel_width=0.5)):
    with plt.rc_context({"legend.loc": "lower center"}):
        fig, ax = plt.subplots(nrows=2, sharex=True)

        plotter.plot_belief(
            ax=ax[0],
            u=u_cond_pde_bc,
            conditioned_on=["pde", "bc"],
            X_pde=X_pde,
            boundary_condition=bvp.boundary_conditions[0],
            simplified_labels=True,
        )

        plotter.plot_pred_belief(
            ax=ax[1],
            u=u_cond_pde_bc,
            conditioned_on=["pde", "bc"],
            X_pde=X_pde,
            f_X_pde=f_X_pde,
            simplified_labels=True,
        )
    
experiment_utils.savefig("02_cond_pde_bc")

### Condition on Interior Measurements

In [ ]:
X_dts = X_core_centers + 0.2
u_X_dts = np.array([60.6, 60.8, 60])
u_X_dts_noise = pn.randvars.Normal(
    mean=np.zeros_like(u_X_dts),
    cov=0.3 ** 2 * np.eye(u_X_dts.size),
)

In [ ]:
u_cond_pde_dts = u_cond_pde.condition_on_observations(
    X=X_dts,
    Y=u_X_dts,
    b=u_X_dts_noise,
)

In [ ]:
plotter.plot_belief(
    ax=plt.gca(),
    u=u_cond_pde_dts,
    conditioned_on=["pde", "dts"],
    X_pde=X_pde,
    X_dts=X_dts,
    u_X_dts=u_X_dts + u_X_dts_noise,
)

In [ ]:
plotter.plot_pred_belief(
    ax=plt.gca(),
    u=u_cond_pde_dts,
    conditioned_on=["pde", "dts"],
    X_pde=X_pde,
    f_X_pde=f_X_pde,
)

In [ ]:
with plt.rc_context(config.tueplots_bundle(nrows=2, rel_width=0.5)):
    with plt.rc_context({"legend.loc": "lower center"}):
        fig, ax = plt.subplots(nrows=2, sharex=True)

        plotter.plot_belief(
            ax=ax[0],
            u=u_cond_pde_dts,
            conditioned_on=["pde", "dts"],
            X_pde=X_pde,
            X_dts=X_dts,
            u_X_dts=u_X_dts + u_X_dts_noise,
            simplified_labels=True,
        )

        plotter.plot_pred_belief(
            ax=ax[1],
            u=u_cond_pde_dts,
            conditioned_on=["pde", "dts"],
            X_pde=X_pde,
            f_X_pde=f_X_pde,
            simplified_labels=True,
        )

experiment_utils.savefig("03_cond_pde_dts")

### Neumann Boundary Conditions

According to Fourier's law, the outward heat flux through the surface of the CPU is
given by $q(x) = \langle n(x), - k \nabla u(x) \rangle = - k \nabla_{n(x)} u(x)$.

In [ ]:
g_neumann = np.full(
    (2,),
    TDP_CPU / A_CPU_TI
)

eps_neumann = pn.randvars.Normal(
    np.zeros_like(g_neumann),
    np.diag(np.full_like(g_neumann, 0.2 ** 2))
)

In [ ]:
left_boundary_op = -k_CPU * linpde_gp.linfuncops.diffops.DirectionalDerivative(-1.0)
right_boundary_op = -k_CPU * linpde_gp.linfuncops.diffops.DirectionalDerivative(1.0)

In [ ]:
u_cond_pde_dts_bc = u_cond_pde_dts.condition_on_observations(
    X=[domain[0]],
    Y=[g_neumann[0]],
    L=left_boundary_op,
    b=eps_neumann[[0]],
)

u_cond_pde_dts_bc = u_cond_pde_dts_bc.condition_on_observations(
    X=[domain[1]],
    Y=[g_neumann[1]],
    L=right_boundary_op,
    b=eps_neumann[[1]],
)

In [ ]:
plotter.plot_belief(
    ax=plt.gca(),
    u=u_cond_pde_dts_bc,
    conditioned_on=["pde", "dts", "nbc"],
    X_pde=X_pde,
    X_dts=X_dts,
    u_X_dts=u_X_dts + u_X_dts_noise,
    boundary_condition=g_neumann / -k_CPU + eps_neumann,
)

In [ ]:
plotter.plot_pred_belief(
    ax=plt.gca(),
    u=u_cond_pde_dts,
    conditioned_on=["pde", "dts", "nbc"],
    X_pde=X_pde,
    f_X_pde=f_X_pde,
)

In [ ]:
with plt.rc_context(config.tueplots_bundle(nrows=2, rel_width=0.5)):
    with plt.rc_context({"legend.loc": "lower center"}):
        fig, ax = plt.subplots(nrows=2, sharex=True)

        plotter.plot_belief(
            ax=ax[0],
            u=u_cond_pde_dts_bc,
            conditioned_on=["pde", "dts", "nbc"],
            X_pde=X_pde,
            X_dts=X_dts,
            u_X_dts=u_X_dts + u_X_dts_noise,
            boundary_condition=g_neumann / -k_CPU + eps_neumann,
            simplified_labels=True,
        )

        plotter.plot_pred_belief(
            ax=ax[1],
            u=u_cond_pde_dts,
            conditioned_on=["pde", "dts", "nbc"],
            X_pde=X_pde,
            f_X_pde=f_X_pde,
            simplified_labels=True,
        )

experiment_utils.savefig("04_cond_pde_dts_nbc")

## Full Model

\begin{align*}
    u & \sim \mathcal{GP}(m_u, k_u) \\
    \dot{q}_V & \sim \mathcal{GP}(m_f, k_f) \\
    \dot{q}_A & \sim \mathcal{N}(\mu_g, \Sigma_u) \\
\end{align*}

\begin{align*}
    w_\text{CPU} \cdot d_\text{CPU} \cdot \int_D \dot{q}_V(x) \mathrm{d}x - w_\text{CPU} \cdot d_\text{CPU} \cdot \left( \dot{q}_{A, 1} + \dot{q}_{A, 2} \right) - 2 \cdot l_\text{CPU} \cdot d_\text{CPU} \cdot \dot{q}_{A, 3} & = 0 \\
    -k \Delta u(x_{\text{PDE}, i}) - \dot{q}_V(x_{\text{PDE}, i}) & = 0 \\
    \frac{\partial u(x_{\text{NBC}, i})}{\partial \nu(x_{\text{NBC}, i})} - \dot{q}_{A, i} & = 0 \qquad \qquad \qquad i \in \{ 1, 2 \} \\
    u(x_{\text{DTS}, i}) + \epsilon_{\text{DTS}, i} & = \hat{u}_{\text{DTS}, i}
\end{align*}

### Prior

In [ ]:
u_prior = pn.randprocs.GaussianProcess(
    mean=linpde_gp.functions.Constant(input_shape=(), value=59.0),
    cov=3.0 ** 2 * linpde_gp.randprocs.kernels.Matern(
        input_shape=(),
        p=3,
        lengthscale=8.0,
    ),
)

In [ ]:
f_prior = pn.randprocs.GaussianProcess(
    mean=f_CPU - linpde_gp.functions.Constant((), TDP_CPU / A_CPU_TI / d_CPU),
    cov=linpde_gp.randprocs.kernels.Matern(
        input_shape=(),
        p=3,
        lengthscale=8.0,
    ),
)

In [ ]:
g_prior = pn.randvars.Normal(
    mean=np.full((3,), TDP_CPU / A_CPU_TI),
    cov=0.2 ** 2 * np.eye(3),
)

### Stationarity Condition

In [ ]:
L_stat = w_CPU * d_CPU * linpde_gp.linfuncops.UndefinedLebesgueIntegral(lower_bound=domain[0])
A_stat = -np.array([A_CPU_EW, A_CPU_EW, 2 * A_CPU_NS])

f_stat = f_prior.condition_on_observations(
    X=[domain[1]],
    Y=[0.0],
    L=L_stat,
    b=A_stat @ g_prior,
)

g_stat = g_prior.condition_on_observations(
    observations=0.0,
    noise=L_stat(f_prior)(domain[1]),
    transform=A_stat,
)

In [ ]:
f_stat.plot(
    plt.gca(), np.linspace(0.0, l_CPU), num_samples=10, rng=np.random.default_rng(24)
)